In [2]:
import numpy as np
import cv2
import os
import json

def create2DArray(start_val, end_val):
    chess_2d_array = []
    val = np.arange(start_val,end_val)
    for a in val:
        for b in val:
            chess_2d_array.append([a,b])
    return chess_2d_array

#corners: quello che ti dà la findChessboardCorners
def saveToJSON(dict, camera_number):
    with open(f"{camera_number}Fcorners.json", "a+") as outfile:
        json.dump(dict, outfile)
        return True
    return False

In [9]:
camera_number = 8
frame_count = 0
image_number = 10

#print(os.listdir())

abs_dir_path = "videos/"
VIDEO_NAME = f'out{camera_number}F.mp4'
ROW_POINTS = 5
COLUMNS_POINTS = 7
# Open the video file 
video_capture = cv2.VideoCapture(filename=f'{abs_dir_path}{VIDEO_NAME}')

# Print the number of frames in the video
number_of_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", number_of_frame)
#superior level frame_skip
frame_skip = number_of_frame / image_number
#sub level frame skip 10 frames
sub_skip = 10

Number of frames in the video:  3678


In [3]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + VIDEO_NAME
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))

os.chdir(output_dir)



Saving frames to  samples/out1F.mp4


IsADirectoryError: [Errno 21] Is a directory: 'samples/out1F.mp4/samples'

In [10]:
##JSON TEST
frame_skip = 10
for i in range(1400):
    ret, img = video_capture.read()

json_dict = {}

while True:
        # Read a frame from the video
        ret, img = video_capture.read()
        if not ret:
            break  # Break the loop if we've reached the end of the video

        # Skip frames based on the frame_skip value
        if frame_count % frame_skip == 0 :
            # Resize the frame to the new_resolution
            real_img = img
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #corners = []
            # Find the chessboard corners
            found_corners, corners = cv2.findChessboardCorners(gray, (ROW_POINTS, COLUMNS_POINTS), corners= None)
            #res_img = cv2.drawChessboardCorners(img,(ROW_POINTS,COLUMNS_POINTS),corners,found_corners)
            if found_corners:
                #populating the dictionary with the points
                print('saving to JSON')
                json_dict.update([[f"{camera_number}_{1000 + frame_count}" , corners.tolist()]])
            print(frame_count)
        
            #print("Frame ", frame_count, ":", found_corners)
            #res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR)
        frame_count += 1  
        if frame_count == 300:
            break  

res = saveToJSON(json_dict, camera_number)
print(res)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
saving to JSON
140
150
160
170
180
190
200
210
220
saving to JSON
230
240
250
260
270
280
290
True


In [ ]:
video_capture = cv2.VideoCapture(f'{abs_dir_path}{VIDEO_NAME}')

FRAME_NUMBER = 500
frame_count = 0
img = 0
while frame_count != FRAME_NUMBER:
    ret, img = video_capture.read()
    frame_count += 1

res_img = img

cv2.namedWindow(f'{FRAME_NUMBER}', flags= cv2.WINDOW_NORMAL)
cv2.resizeWindow(f'{FRAME_NUMBER}', 1280, 720)
# cv2.imshow(f'{FRAME_NUMBER}', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

found_config = []
found_corners = []

np.arange(1,8)

print(create2DArray(3,8))

for a,b in create2DArray(5,8):
    print(f"{a}, {b}")
    ret, corners = cv2.findChessboardCorners(gray, (a, b), None)
    if ret == True:
        print("found one")
        found_config.append([a,b])
        found_corners.append([[a,b],corners])

print(f"configurazioni trovate: {found_config}")

cv2.drawChessboardCorners(res_img,(found_config[0][0],found_config[0][1]), found_corners[0][1], True)
cv2.imshow(f'{FRAME_NUMBER}', res_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_path_name = f'{found_config[0]}.jpg'
cv2.imwrite(img_path_name, res_img)

In [22]:
saveToJSON(np.array([[12,3],[3,3]]),1,210)
saveToJSON(np.array([[12,3],[3,3]]),2,100)
